In [170]:
import agate

In [171]:
tester = agate.TypeTester(limit=500)

In [172]:
crime = agate.Table.from_csv('oncampuscrime111213.csv', column_types=tester)

Creating a table of Missouri Schools 

In [173]:
mo_crime = crime.where(lambda r: r['State'] == 'MO')
print len(mo_crime.rows)


325


Creating two tables to look at public, private MO schools separately

In [174]:
mo_public = mo_crime.where(lambda r: r['Sector_desc'].startswith('Pub'))
print len(mo_public.rows) 

99


In [175]:
mo_private = mo_crime.where(lambda r: r['Sector_desc'].startswith('Pri'))
print len(mo_private.rows) 

226


Question: How many violent crimes occurred on public and private campuses in 2013?

In [176]:
v1 = mo_public.aggregate(agate.Sum('MURD13'))
v2 = mo_public.aggregate(agate.Sum('NEG_M13'))
v3 = mo_public.aggregate(agate.Sum('FORCIB13'))
v4 = mo_public.aggregate(agate.Sum('NONFOR13'))
v5 = mo_public.aggregate(agate.Sum('ROBBE13'))
v6 = mo_public.aggregate(agate.Sum('AGG_A13'))
v7 = mo_public.aggregate(agate.Sum('BURGLA13'))
v8 = mo_public.aggregate(agate.Sum('ARSON13'))

vc_13 = v1 + v2 + v3 + v4 + v5 + v6 + v7 + v8

print vc_13

276


In [177]:
v1 = mo_private.aggregate(agate.Sum('MURD13'))
v2 = mo_private.aggregate(agate.Sum('NEG_M13'))
v3 = mo_private.aggregate(agate.Sum('FORCIB13'))
v4 = mo_private.aggregate(agate.Sum('NONFOR13'))
v5 = mo_private.aggregate(agate.Sum('ROBBE13'))
v6 = mo_private.aggregate(agate.Sum('AGG_A13'))
v7 = mo_private.aggregate(agate.Sum('BURGLA13'))
v8 = mo_private.aggregate(agate.Sum('ARSON13'))

vc_13 = v1 + v2 + v3 + v4 + v5 + v6 + v7 + v8

print vc_13

260


536 violent crimes occurred on campuses, only slightly more on private campuses. What's the population on these campuses?

In [178]:
public_schools = mo_public.distinct('INSTNM')
public_pop = public_schools.aggregate(agate.Sum('Total'))
print public_pop

256491


In [179]:
private_schools = mo_private.distinct('INSTNM')
private_pop = private_schools.aggregate(agate.Sum('Total'))
print private_pop

243464


Well, that didn't tell me much. I expected a larger population on public schools ... 

In [180]:
group_pop = mo_public.group_by('INSTNM')

sb_pop = group_pop.aggregate([
        ('population', agate.Sum('Total'))
    ])

public_campus_pop = sb_pop.order_by('population', reverse=True)

public_campus_pop.print_table(max_rows=20)


|------------------------------------------------+-------------|
|  INSTNM                                        | population  |
|------------------------------------------------+-------------|
|  Saint Louis Community College                 |    216,045  |
|  Metropolitan Community College-Kansas City    |    115,404  |
|  Ozarks Technical Community College            |     73,990  |
|  Southeast Missouri State University           |     47,068  |
|  Moberly Area Community College                |     39,970  |
|  University of Missouri-Columbia               |     34,616  |
|  University of Missouri-Kansas City            |     31,436  |
|  University of Central Missouri                |     25,026  |
|  Crowder College                               |     23,380  |
|  Missouri State University-Springfield         |     21,271  |
|  Jefferson College                             |     20,776  |
|  Northwest Missouri State University           |     19,455  |
|  University of Missouri

Community colleges seem to account for a large percentage of the public school population. I'll remove the community colleges.

In [181]:
find_community = public_campus_pop.order_by('INSTNM')

for row in public_campus_pop.rows:
    print(row['INSTNM'])

Saint Louis Community College
Metropolitan Community College-Kansas City
Ozarks Technical Community College
Southeast Missouri State University
Moberly Area Community College
University of Missouri-Columbia
University of Missouri-Kansas City
University of Central Missouri
Crowder College
Missouri State University-Springfield
Jefferson College
Northwest Missouri State University
University of Missouri-St Louis
East Central College
Truman State University
Missouri University of Science and Technology
St Charles Community College
Lincoln University
Missouri Western State University
Missouri Southern State University
State Fair Community College
Mineral Area College
Three Rivers Community College
Missouri State University-West Plains
Linn State Technical College
North Central Missouri College
Harris-Stowe State University
Applied Technology Services
Franklin Technology-MSSU
Cape Girardeau Career and Technology Center
Rolla Technical Institute
Hillyard Technical Center
South Central Career 

eh, maybe not. Same issue with career, medical and techical centers ...

... I tried to create a new column of violent crime totals using Formula http://bit.ly/1QztYpa and Excel formula http://bit.ly/1Qzu0gO, but couldn't figure it out. I decided to compare sex offenses at MU versus comparable MO public schools instead.

In [182]:
mo_compare = mo_public.where(lambda r: r['INSTNM'].startswith('Southeast Missouri') or r['INSTNM'].startswith('University of') or r['INSTNM'].startswith('Missouri State') or r['INSTNM'].startswith('Northwest'))


In [183]:
for row in mo_compare.rows: 
    print(row['INSTNM'])

University of Central Missouri
University of Central Missouri
University of Missouri-Columbia
University of Missouri-Kansas City
University of Missouri-Kansas City
University of Missouri-St Louis
Northwest Missouri State University
Northwest Missouri State University
Northwest Missouri State University
Missouri State University-West Plains
Missouri State University-West Plains
Southeast Missouri State University
Southeast Missouri State University
Southeast Missouri State University
Southeast Missouri State University
Missouri State University-Springfield


In [184]:
group_pop = mo_compare.group_by('INSTNM')

sb_pop = group_pop.aggregate([
        ('population', agate.Sum('Total')),
        ('forcible', agate.Sum('FORCIB13')),
        ('nonforcible', agate.Sum('NONFOR13'))
    ])

public_campus_pop = sb_pop.order_by('population', reverse=True)

public_campus_pop.print_table()



|----------------------------------------+------------+----------+--------------|
|  INSTNM                                | population | forcible | nonforcible  |
|----------------------------------------+------------+----------+--------------|
|  Southeast Missouri State University   |     47,068 |        7 |           0  |
|  University of Missouri-Columbia       |     34,616 |        7 |           0  |
|  University of Missouri-Kansas City    |     31,436 |        0 |           0  |
|  University of Central Missouri        |     25,026 |        6 |           0  |
|  Missouri State University-Springfield |     21,271 |        6 |           0  |
|  Northwest Missouri State University   |     19,455 |        4 |           0  |
|  University of Missouri-St Louis       |     16,809 |        1 |           0  |
|  Missouri State University-West Plains |      4,246 |        0 |           0  |
|----------------------------------------+------------+----------+--------------|


In [185]:
group_pop = mo_compare.group_by('INSTNM')

sb_pop = group_pop.aggregate([
        ('population', agate.Sum('Total')),
        ('forcible', agate.Sum('FORCIB12')),
        ('nonforcible', agate.Sum('NONFOR12'))
    ])

public_campus_pop = sb_pop.order_by('population', reverse=True)

public_campus_pop.print_table()


|----------------------------------------+------------+----------+--------------|
|  INSTNM                                | population | forcible | nonforcible  |
|----------------------------------------+------------+----------+--------------|
|  Southeast Missouri State University   |     47,068 |        2 |           0  |
|  University of Missouri-Columbia       |     34,616 |        6 |           0  |
|  University of Missouri-Kansas City    |     31,436 |        1 |           0  |
|  University of Central Missouri        |     25,026 |        8 |           0  |
|  Missouri State University-Springfield |     21,271 |        3 |           0  |
|  Northwest Missouri State University   |     19,455 |        2 |           0  |
|  University of Missouri-St Louis       |     16,809 |        2 |           0  |
|  Missouri State University-West Plains |      4,246 |        1 |           0  |
|----------------------------------------+------------+----------+--------------|


In [186]:
group_pop = mo_compare.group_by('INSTNM')

sb_pop = group_pop.aggregate([
        ('population', agate.Sum('Total')),
        ('forcible', agate.Sum('FORCIB11')),
        ('nonforcible', agate.Sum('NONFOR11'))
    ])

public_campus_pop = sb_pop.order_by('population', reverse=True)

public_campus_pop.print_table()


|----------------------------------------+------------+----------+--------------|
|  INSTNM                                | population | forcible | nonforcible  |
|----------------------------------------+------------+----------+--------------|
|  Southeast Missouri State University   |     47,068 |        5 |           0  |
|  University of Missouri-Columbia       |     34,616 |       11 |           0  |
|  University of Missouri-Kansas City    |     31,436 |        1 |           0  |
|  University of Central Missouri        |     25,026 |        5 |           0  |
|  Missouri State University-Springfield |     21,271 |        0 |           0  |
|  Northwest Missouri State University   |     19,455 |        2 |           0  |
|  University of Missouri-St Louis       |     16,809 |        0 |           0  |
|  Missouri State University-West Plains |      4,246 |        0 |           0  |
|----------------------------------------+------------+----------+--------------|


Story idea: Missouri State University-Springfield had the highest rate of on-campus sex offenses among the state's five largest four-year public universities in 2013. Annual cases at the university are rising as well, up from two forcible sex offenses in 2012 and zero in 2011. That may have something to do with campus population trends, but it's unclear from the data.

The university reported six cases of on-campus offenses out of a student body population of 25,026. That's about double the rate of incidents reported at Southeast Missouri State University, which had seven cases among 47,068 students.  

On the other hand, the University of Missouri-Kansas City had two on-campus sex offenses in three years, which is interesting considering its in a large city; the University of Missouri-St Louis also reported a low number of on-campus incidents over the three years in comparison.

The annual rate reported at each school fluctuates drastically each year due to the mostly single-digit numbers, so it's hard to draw a conclusion about the safety of female students based on one year. There does seem to be a steady climb at Missouri State University-Springfield, however. I'd be curious to know what's causing the increase. Is it simply a population factor or something else?  